In [4]:
import tensorflow as tf
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_dir = '/content/drive/MyDrive/UNRC/Tercer Año/Proyecto/chest_xray/train'  # Ruta de entrenamiento

# Data augmentation para mejorar el rendimiento del modelo.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

Generamos lotes de imágenes desde el directorio, aplicando algunas transformaciones y preprocesamiento básico antes de alimentar las imágenes al modelo durante el entrenamiento

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Tamaño de las imágenes
    batch_size=32,          #Tamaño de lote
    class_mode='categorical',  # Porque es un problema de clasificación binaria
    subset='training'     # Usar como conjunto de entrenamiento
)

Found 2146 images belonging to 2 classes.


Hacemos lo mismo para el conjunto de validación...

In [7]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 536 images belonging to 2 classes.


In [8]:
# Creamos el modelo
model = models.Sequential()

In [9]:
# Capas de convulucion y pooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))  # 3 canales de entrada (RGB)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

In [10]:
# Capas densas
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [11]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Una vez, definido todo lo que es el modelo, comenzamos a entrenarlo con el conjunto de datos

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/30
 3/67 [>.............................] - ETA: 4:36 - loss: 0.9542 - accuracy: 0.5417

Ya entrenado, procedemos a hacer uso del conjunto de prueba, para validarlo...

In [ ]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Loss: {val_loss}, Accuracy: {val_acc}')

In [ ]:
model.save('/content/drive/MyDrive/UNRC/Tercer Año/Proyecto/modelo-xray-jere.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image

def predict_image(img_path, model):
  img = image.load_img(img_path, target_size=(150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x /= 255.

  classes = model.predict(x)
  result = classes[0]

  # Obtener el nombre de la clase real a partir de la ruta de la imagen
  true_class = img_path.split('/')[-2]

  # Obtener los porcentajes para cada clase
  pneumonia_percentage = result[0] * 100
  normal_percentage = result[1] * 100

  # Mostrar resultados
  print(f"Resultado esperado: {true_class}")
  print(f"Resultado obtenido: {pneumonia_percentage:.2f}% PNEUMONIA, {normal_percentage:.2f}% NORMAL")

In [ ]:
predict_image('/content/drive/MyDrive/UNRC/Tercer Año/Proyecto/chest_xray/test/PNEUMONIA/person100_bacteria_480.jpeg', model)